In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
import re
osp = sys.platform
# import matplotlib.pyplot as plt
# %matplotlib inline

from utilities import PlotGame, LoadData, make_video
from helpers import id_player, id_position, id_teams

# ---------------------------------------------------------
%load_ext autoreload
%autoreload 2

In [2]:
# directories
main_dir = '../'
game_dir = main_dir+'data/'
# court_path = data_dir + 'nba_court_T.png'

file_names = glob.glob(game_dir+'*.pkl')
gameids = [re.findall('\d+', i)[0] for i in file_names]
gameids = sorted(gameids, key=lambda x: int(x))

In [3]:
def create_event(gameid, event_number, video=True):    
    Data = LoadData(main_dir, game_dir)
    data = Data.load_game(gameid)
    
    # get events and number of events
    event_df = pd.DataFrame(data['events'])
    team_map = id_teams([event_df])
    
    # get the mean of the team
    # use the first moment from first moment
    init_moment = np.array(event_df['moments'][0][0][5])
#     print(init_moment)
    assert len(init_moment) == 11
    t1 = init_moment[1:6, :]
    t2 = init_moment[6:, :]
    tx_mean1 = np.mean(t1[:, 2], axis=0)
    tx_mean2 = np.mean(t2[:, 2], axis=0)
    court_position = -1
    oteam = 'none'
    if tx_mean1 > tx_mean2:
        court_position = 1
        oteam = team_map[t2[0][0]]
    else:
        court_position = 0
        oteam = team_map[t1[0][0]]
        
    print('court position >>> {0:} | team >>> {1:}'.format(court_position, oteam))
    n_events = event_df.shape[0]
    # get numebr of moments
    moments = event_df.iloc[event_number,:]['moments']
    n_moments = len(moments)

    print(event_df.shape, n_moments)
#     event_df.head()
    
    Plot = PlotGame(gameid, main_dir, game_dir)
    # save all moments for a specific events
    moments = range(n_moments)
    for i in moments:
        print(i, end='\r')
        Plot.load_moment2img(data, event_number, i)
    #     break
    
    if video:
        # prepare to save videos
        event_dir = game_dir + 'game' + str(gameid) + '/' + 'event' + str(event_number) + '/'
        video_name = event_dir + str(event_number) + '.mp4'
        print(video_name)
        images = glob.glob(event_dir + "*.png")
        if osp == 'linux':
            # this is pretty hard-coded, should change
            images = sorted(images, key=lambda x: int(x.split('.')[2].split('/')[-1]))
        else:
            images = sorted(images, key=lambda x: int(x.split('\\')[-1].split('.')[0]))
        # print(images)
        # save to video
        make_video(images, video_name, 15)
    
    return court_position, oteam
        
# we try to extract which team belongs to the left most
# i.e. (0,0) side
court_positions_id = {}
court_positions_team = {}
# we will create all the event 0 for all game id
for i in gameids:
    print(i, '='*10)
    court_positions_id[i], court_positions_team[i] = create_event(i, -1, video=True)
    break

0021500009 ==========
court position >>> 0 | team >>> toronto raptors
(268, 8) 250
../data/game0021500009/event-1/-1.mp4
The output video is ../data/game0021500009/event-1/-1.mp4


In [9]:
# fix mistakes
# 0021500290: 1, golden state warriors
# 0021500463: 0, chicago bulls
court_positions_id['0021500290'] = 1
court_positions_team['0021500290'] = 'golden state warriors'

court_positions_id['0021500463'] = 0
court_positions_team['0021500463'] = 'chicago bulls'

In [15]:
# save to meta
df = pd.DataFrame.from_dict(court_positions_team, orient='index')
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'game_id', 0: 'court_position'})
df.to_csv('./meta_data/court_name.csv', index=False)

In [ ]:
# # get events and number of events
# event_df = pd.DataFrame(data['events'])
# n_events = event_df.shape[0]
# # get numebr of moments
# moments = event_df.iloc[event_number,:]['moments']
# n_moments = len(moments)

# print(n_moments, event_df.shape)
# event_df.head()

In [ ]:
# h = event_df.home[0]
# h_team, h_id = h['name'], h['teamid']

# v = event_df.visitor[0]
# v_team, v_id = v['name'], v['teamid']

# event_number = 0

In [ ]:
# Plot = PlotGame(gameid, main_dir, game_dir)
# # save all moments for a specific events
# moments = range(n_moments)
# for i in moments:
#     print(i, end='\r')
#     Plot.load_moment2img(data, event_number, i)
# #     break

In [ ]:
# # prepare to save videos
# event_dir = game_dir + 'game' + str(gameid) + '/' + 'event' + str(event_number) + '/'
# video_name = event_dir + str(event_number) + '.mp4'
# print(video_name)
# images = glob.glob(event_dir + "*.png")
# if osp == 'linux':
#     # this is pretty hard-coded, should change
#     images = sorted(images, key=lambda x: int(x.split('.')[2].split('/')[-1]))
# else:
#     images = sorted(images, key=lambda x: int(x.split('\\')[-1].split('.')[0]))
# # print(images)
# # save to video
# make_video(images, video_name, 15)

In [ ]:
event_df.loc[0,:].playbyplay

In [ ]:
event_df.playbyplay[0].to_dict(orient='records')

In [ ]:
event_df.loc[0, :].moments

In [ ]:
# event_df

In [ ]:
id_team = id_teams([event_df])
id_team

In [ ]:
hid = 1610612741
vid = 1610612761
hteam_state = []
vteam_state = []
for i in event_df.loc[0, :].moments:
    ai = np.array(i[5])
    assert len(ai) == 11
    bi = np.array(i[5][0][2:4])
    ha = ai[ai[:, 0]==hid, 2:4]
    va = ai[ai[:, 0]==vid, 2:4]
#     print(bi, ha, va)
    hd = (np.sum((ha-bi)**2, axis=1)**0.5).min()
    vd = (np.sum((va-bi)**2, axis=1)**0.5).min()
    if hd > vd:
        hteam_state.append('defend')
        vteam_state.append('offend')
    else:
        hteam_state.append('offfend')
        vteam_state.append('defend')
#     break

In [ ]:
# hteam_state

In [ ]:
a = np.array([ [1,2], [0,3] ])
b = np.array([2, 3])
a

In [ ]:
b

In [ ]:
a - b

In [ ]:
(a-b)**2

In [ ]:
np.sum((a-b)**2, axis=1)**0.5